# RAPIDS cuML TfidfVectorizer and KNN to find similar Text and Images
In this notebook we use RAPIDS cuML's TfidfVectorizer and cuML's KNN to find items with similar titles and items with similar images. First we use RAPIDS cuML TfidfVectorizer to extract text embeddings of each item's title and then compare the embeddings using RAPIDS cuML KNN. Next we extract image embeddings of each item with EffNetB0 and compare them using RAPIDS cuML KNN.[](http://)

# Load Libraries

In [ ]:
import pip
try:
     __import__("imutils")
except ImportError:
    pip.main(["install", "imutils"])

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import  matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import cv2
from imutils.object_detection import non_max_suppression
import pytesseract
import os
import glob
os.chdir('/kaggle/working')
BASE = '../input/shopee-product-matching/train_images/'
EAST_PATH = "/kaggle/input/east-data-leo/frozen_east_text_detection/frozen_east_text_detection.pb"
RESIZE_IMG_SHAPE = (500,500)
img_paths = glob.glob(f"{BASE}*jpg") 

# Load Train Data
In this competition, we have items with an image and title. For the train data, the column `label_group` indicates the ground truth of which items are similar. We need to build a model that finds these similar images based on their image and title's text. In this notebook we explore some tools to help us.

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')
print('train shape is', train.shape )
train.head()

# Display Duplicated Items from Train Data
Using the column `label_group` which is the ground truth, we can display examples of duplicated items.

In [ ]:
groups = train.label_group.value_counts()
plt.figure(figsize=(20,5))
plt.plot(np.arange(len(groups)),groups.values)
plt.ylabel('Duplicate Count',size=14)
plt.xlabel('Index of Unique Item',size=14)
plt.title('Duplicate Count vs. Unique Item Count',size=16)
plt.show()

plt.figure(figsize=(20,5))
plt.bar(groups.index.values[:50].astype('str'),groups.values[:50])
plt.xticks(rotation = 45)
plt.ylabel('Duplicate Count',size=14)
plt.xlabel('Label Group',size=14)
plt.title('Top 50 Duplicated Items',size=16)
plt.show()

In [ ]:
def img_proc(img,offset =20):
    img =img[offset:-offset,offset:-offset]
    img =cv2.resize(img ,RESIZE_IMG_SHAPE)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray,5)
    ret, thresh = cv2.threshold(gray,150,255,cv2.THRESH_BINARY)
    thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY_INV,21,2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    opening = cv2.erode(thresh,kernel, iterations = 1)
    closing = cv2.dilate(opening,kernel,iterations=1)
    contours = cv2.findContours(closing, 
                            cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_NONE)
    canvas =np.zeros((500,500))
    cv2.drawContours(canvas, contours[0], -1, (255,0,255), thickness = 2) 
    return canvas



def displayDF(train, random=False, COLS=6, ROWS=4, path=BASE):
    for k in range(ROWS):
        plt.figure(figsize=(20,5))
        for j in range(COLS):
            if random: row = np.random.randint(0,len(train))
            else: row = COLS*k + j
            name = train.iloc[row,1]
            title = train.iloc[row,3]
            title_with_return = ""
            for i,ch in enumerate(title):
                title_with_return += ch
                if (i!=0)&(i%20==0): title_with_return += '\n'
            img = img_proc(cv2.imread(path+name))
            plt.subplot(1,COLS,j+1)
            plt.title(title_with_return)
            plt.axis('off')
            plt.imshow(img)
        plt.show()
        
displayDF(train,random=True)

In [ ]:
def predictions(prob_score, geo,min_conf):
    (numR, numC) = prob_score.shape[2:4]
    boxes = []
    confidence_val = []
    for y in range(0, numR):
        scoresData = prob_score[0, 0, y]
        x0 = geo[0, 0, y]
        x1 = geo[0, 1, y]
        x2 = geo[0, 2, y]
        x3 = geo[0, 3, y]
        anglesData = geo[0, 4, y]
        for i in range(0, numC):
            if scoresData[i] < min_conf:
                continue
            (offX, offY) = (i * 4.0, y * 4.0)
            angle = anglesData[i]
            cos = np.cos(angle)
            sin = np.sin(angle)
            h = x0[i] + x2[i]
            w = x1[i] + x3[i]
            endX = int(offX + (cos * x1[i]) + (sin * x2[i]))
            endY = int(offY - (sin * x1[i]) + (cos * x2[i]))
            startX = int(endX - w)
            startY = int(endY - h)
            boxes.append((startX, startY, endX, endY))
            confidence_val.append(scoresData[i])

    return (boxes, confidence_val)

def get_text(img,img_text ='Output',min_conf = 0.7, east = EAST_PATH):
    image = cv2.imread(img)
    orig = image.copy()
    (origH, origW) = image.shape[:2]
    (newW, newH) = (320,320)
    rW = origW / float(newW)
    rH = origH / float(newH)
    image = cv2.resize(image, (newW, newH))
    (H, W) = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (W, H),
        (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net = cv2.dnn.readNet(east)
    # The following two layer need to pulled from EAST model for achieving this. 
    layerNames = [
        "feature_fusion/Conv_7/Sigmoid",
        "feature_fusion/concat_3"]

    #Forward pass the blob from the image to get the desired output layers
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)
    (boxes, confidence_val) = predictions(scores, geometry,min_conf)
    boxes = non_max_suppression(np.array(boxes), probs=confidence_val)
    
    results = []
    for (startX, startY, endX, endY) in boxes:
        startX = int(startX * rW)
        startY = int(startY * rH)
        endX = int(endX * rW)
        endY = int(endY * rH)
        r = orig[startY:endY, startX:endX]
        configuration = ("-l eng --oem 1 --psm 8")
        text = pytesseract.image_to_string(r, config=configuration)
        results.append(((startX, startY, endX, endY), text))

    orig_image = orig.copy()
    txts=[]
    for ((start_X, start_Y, end_X, end_Y), text) in results:
        text = "".join([x if ord(x) < 128 else "" for x in text]).strip()
        cv2.rectangle(orig_image, (start_X, start_Y), (end_X, end_Y),
            (0, 0, 255), 2)
        cv2.putText(orig_image, text, (start_X, start_Y + 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0,255, 0), 4)
        txts.append(text)
    plt.figure(figsize=(10,10))
    plt.imshow(orig_image)
    plt.title(img_text)
    plt.show()
    print(txts)

In [ ]:
it = iter(range(500))

In [ ]:
row = next(it)
path = BASE + train.iloc[row,1]
title = train.iloc[row,3]
get_text(path,title,min_conf=0.9)

# Find Similar Titles with RAPIDS!
We will now ignore the ground truth and try to find similar items in train data using only the title's text. First we will extract text embeddings using RAPIDS cuML's TfidfVectorizer. This will turn every title into a one-hot-encoding of the words present. We will then compare one-hot-encodings with RAPIDS cuML KNN to find title's that are similar.

In [ ]:
!pip install cudf

In [ ]:
import cudf, cuml, cupy
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.neighbors import NearestNeighbors
print('RAPIDS',cuml.__version__)

In [ ]:
# LOAD TRAIN UNTO THE GPU WITH CUDF
train_gf = cudf.read_csv('../input/shopee-product-matching/train.csv')
print('train shape is', train_gf.shape )
train_gf.head()

## Extract Text Embeddings with RAPIDS TfidfVectorizer
TfidfVectorizer returns a cupy sparse matrix. Afterward we convert to a cupy dense matrix and feed that into RAPIDS cuML KNN.

In [ ]:
model = TfidfVectorizer(stop_words='english', binary=True)
text_embeddings = model.fit_transform(train_gf.title).toarray()
print('text embeddings shape is',text_embeddings.shape)

## Find Similar Titles with RAPIDS KNN
After fitting KNN, we will display some example rows of train and their 10 closest other titles in train (based on word count one-hot-encoding).

In [ ]:
KNN = 50
model = NearestNeighbors(n_neighbors=KNN)
model.fit(text_embeddings)
distances, indices = model.kneighbors(text_embeddings)

In [ ]:
for k in range(5):
    plt.figure(figsize=(20,3))
    plt.plot(np.arange(50),cupy.asnumpy(distances[k,]),'o-')
    plt.title('Text Distance From Train Row %i to Other Train Rows'%k,size=16)
    plt.ylabel('Distance to Train Row %i'%k,size=14)
    plt.xlabel('Index Sorted by Distance to Train Row %i'%k,size=14)
    plt.show()
    
    print( train_gf.loc[cupy.asnumpy(indices[k,:10]),['title','label_group']] )

# Find Matching Images with RAPIDS!
We will now ignore the ground truth and try to find similar items in train data using only the item's image. First we will extract image embeddings using EffNetB0. We will then compare image embeddings with RAPIDS cuML KNN to find images that are similar.

## Extract Image Embeddings with EffNetB0

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, df, img_size=256, batch_size=32, path=BASE): 
        self.df = df
        self.img_size = img_size
        self.batch_size = batch_size
        self.path = path
        self.indexes = np.arange( len(self.df) )
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = len(self.df) // self.batch_size
        ct += int(( (len(self.df)) % self.batch_size)!=0)
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X = self.__data_generation(indexes)
        return X
            
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' 
        X = np.zeros((len(indexes),self.img_size,self.img_size,3),dtype='float32')
        df = self.df.iloc[indexes]
        for i,(index,row) in enumerate(df.iterrows()):
            img = cv2.imread(self.path+row.image)
            X[i,] = cv2.resize(img,(self.img_size,self.img_size)) #/128.0 - 1.0
        return X

In [ ]:
model = EfficientNetB0(weights='imagenet',include_top=False, pooling='avg', input_shape=None)
train_gen = DataGenerator(train, batch_size=128)
image_embeddings = model.predict(train_gen,verbose=1)
print('image embeddings shape is',image_embeddings.shape)

## Find Similar Images with RAPIDS KNN
After fitting KNN, we will display some example rows of train and their 8 closest other images in train (based EffNetB0 image embeddings).

In [ ]:
KNN = 50
model = NearestNeighbors(n_neighbors=KNN)
model.fit(image_embeddings)
distances, indices = model.kneighbors(image_embeddings)

In [ ]:
for k in range(180,190):
    plt.figure(figsize=(20,3))
    plt.plot(np.arange(50),cupy.asnumpy(distances[k,]),'o-')
    plt.title('Image Distance From Train Row %i to Other Train Rows'%k,size=16)
    plt.ylabel('Distance to Train Row %i'%k,size=14)
    plt.xlabel('Index Sorted by Distance to Train Row %i'%k,size=14)
    plt.show()
    
    cluster = train.loc[cupy.asnumpy(indices[k,:8])] 
    displayDF(cluster, random=False, ROWS=2, COLS=4)

# Test Data
The next thing to do is apply the above methods on the test data and create a submission notebook! Have fun!

In [ ]:
test = pd.read_csv('../input/shopee-product-matching/test.csv')
print( test.shape )
test.head()

In [ ]:
sub = pd.read_csv('../input/shopee-product-matching/sample_submission.csv')
print( sub.shape )
sub.head()